In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
pd.set_option("Display.max_colwidth",200)

In [10]:
users = pd.read_csv("Data/users.csv").rename(columns={
    "User-ID":"user-id",
    "Name":"name",
    "Location":"location",
    "Country":"country",
    "Age":"age"
})
users.head()

,user-id,name,location,country,age
0,1,stockton,california,usa,18
1,2,porto,v.n.gaia,portugal,17
2,3,santa monica,california,usa,61
3,4,albacete,wisconsin,spain,26
4,5,melbourne,victoria,australia,14


In [11]:
users.shape

(100, 5)

In [12]:
ratings = pd.read_csv("Data/ratings.csv").rename(columns={
    "User_ID":"user-id",
    "Food_ID":"food-id",
    "Rating":"rating"
})
ratings.head()

,user-id,food-id,rating
0,1.0,88.0,4.0
1,1.0,46.0,3.0
2,1.0,24.0,5.0
3,1.0,25.0,4.0
4,2.0,49.0,1.0


In [13]:
ratings.shape

(512, 3)

In [14]:
food = pd.read_csv("Data/food.csv").rename(columns={
    "Food_ID":"food-id",
    "Name":"name",
    "C_Type":"type",
    "Veg_Non":"veg-non",
    "Describe":"describe"
})
food.head()

,food-id,name,type,veg-non,describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rind, red chillies, garlic cloves (crushed), olive oil, summer squash (zucchini), sea salt, black pepper, basil leaves"
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), onion, salt, black pepper, carrot, cabbage, green onions, sweet chilli sauce, peanut butter, ginger, soy sauce, fresh cilantro, red pepper flakes (crushe..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, sugar (fine grain), red chilli powder"
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic cloves (minced), chilli pepper (sliced), green papaya, carrot (peeled), cucumbers, mint leaves, toasted peanuts"
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest, lemon zest, jaggery syrup, almond flour, apple, butter (softened), eggs"


In [15]:
food.shape

(400, 5)

In [16]:
user50 = ratings[ratings['user-id']== 50]
user50

,user-id,food-id,rating
262,50.0,19.0,9.0
263,50.0,81.0,8.0
264,50.0,95.0,2.0
265,50.0,57.0,7.0
266,50.0,158.0,6.0
267,50.0,167.0,2.0
268,50.0,225.0,3.0


In [17]:
user50.shape

(7, 3)

In [18]:
ratings_with_food = ratings.merge(food,on="food-id")
ratings_with_food.head()

,user-id,food-id,rating,name,type,veg-non,describe
0,1.0,88.0,4.0,peri peri chicken satay,Snack,non-veg,"boneless skinless chicken thigh (trimmed), salt and pepper, yogurt, chilli powder, ginger garlic paste, coriander leaves, oil to fry, peri peri sauce, potato fries"
1,1.0,46.0,3.0,steam bunny chicken bao,Japanese,non-veg,"buns, all purpose white flour, dry yeast, sugar, salt, warm water, chicken mince, eggs, soy sauce, sugar, sesame oil, worcestershire sauce, chives, aromat powder"
2,1.0,24.0,5.0,green lentil dessert fudge,Dessert,veg,"whole moong beans, cow ghee, raisins, whole milk, jaggery (organic), ground cardamom, almonds (halved)"
3,1.0,25.0,4.0,cashew nut cookies,Dessert,veg,"cashew paste, ghee, khaand (a sweetening agent and a healthier substitute of sugar), flax or chia seeds, plain yogurt, baking soda, baking powder, vanilla, oats, organic all-purpose flour"
4,2.0,49.0,1.0,christmas tree pizza,Italian,veg,"pizza dough (2 boules), red pepper, red onion, basil pesto (purchased, homemade or vegan pesto), mozzarella cheese, kosher salt"


In [19]:
num_ratings = ratings_with_food.groupby('name')['rating'].count().reset_index()
num_ratings.rename(columns={
    'rating':'num of rating'
},inplace=True)
num_ratings.head()

,name,num of rating
0,active charcoal modak,2
1,al hachi chicken,2
2,almond white chocolate gujiya,5
3,almond and amaranth ladoo,3
4,almond and chicken momos (without shell),4


In [20]:
final_ratings = ratings_with_food.merge(num_ratings,on="name")
final_ratings.head()

,user-id,food-id,rating,name,type,veg-non,describe,num of rating
0,1.0,88.0,4.0,peri peri chicken satay,Snack,non-veg,"boneless skinless chicken thigh (trimmed), salt and pepper, yogurt, chilli powder, ginger garlic paste, coriander leaves, oil to fry, peri peri sauce, potato fries",1
1,1.0,46.0,3.0,steam bunny chicken bao,Japanese,non-veg,"buns, all purpose white flour, dry yeast, sugar, salt, warm water, chicken mince, eggs, soy sauce, sugar, sesame oil, worcestershire sauce, chives, aromat powder",5
2,1.0,24.0,5.0,green lentil dessert fudge,Dessert,veg,"whole moong beans, cow ghee, raisins, whole milk, jaggery (organic), ground cardamom, almonds (halved)",3
3,1.0,25.0,4.0,cashew nut cookies,Dessert,veg,"cashew paste, ghee, khaand (a sweetening agent and a healthier substitute of sugar), flax or chia seeds, plain yogurt, baking soda, baking powder, vanilla, oats, organic all-purpose flour",4
4,2.0,49.0,1.0,christmas tree pizza,Italian,veg,"pizza dough (2 boules), red pepper, red onion, basil pesto (purchased, homemade or vegan pesto), mozzarella cheese, kosher salt",6


In [21]:
povit_food = final_ratings.pivot_table(columns = "user-id",index = "name",values = 'rating')
povit_food

user-id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
name,,,,,,,,,,,,,,,,,,,,,
active charcoal modak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
al hachi chicken,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
almond white chocolate gujiya,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN
almond and amaranth ladoo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
almond and chicken momos (without shell),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
watermelon and strawberry smoothie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
white chocolate and lemon pastry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
whole wheat cake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
povit_food.fillna(0,inplace=True)
povit_food

user-id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
name,,,,,,,,,,,,,,,,,,,,,
active charcoal modak,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
al hachi chicken,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
almond white chocolate gujiya,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
almond and amaranth ladoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
almond and chicken momos (without shell),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
watermelon and strawberry smoothie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
white chocolate and lemon pastry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
whole wheat cake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
food_sparse  = csr_matrix(povit_food)
food_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 508 stored elements and shape (309, 100)>

In [24]:
model = NearestNeighbors(algorithm="brute")

In [25]:
model.fit(food_sparse)

NearestNeighbors(algorithm='brute')

In [26]:
def recommemd_food(food_name):
    food_id = np.where(povit_food.index == food_name)[0][0]
    destance,suggestion = model.kneighbors(povit_food.iloc[food_id,:].values.reshape(1,-1),n_neighbors=16)
    for i in range(len(suggestion)):
        suggestion_food = povit_food.index[suggestion[i]]
        
        id = 1
        for j in suggestion_food:
            if j == food_name:
                print("Recommanded to :",j,"\n")
            else:             
                print(f"{id}) {j} \n \n")
                id += 1

In [27]:
food_name ="active charcoal modak"
recommemd_food(food_name)

Recommanded to : active charcoal modak 

1) sweet potato pie 
 

2) chicken tenders 
 

3) chocolate prunes gujiyas 
 

4) chocolate marquise 
 

5) fried rice with soya chunks 
 

6) brown rice 
 

7) chicken dragon 
 

8) holi special bhang pakode 
 

9) garlic soya chicken 
 

10) bhurji- egg 
 

11) egg and garlic fried rice 
 

12) beetroot and green apple soup 
 

13) coconut mango oatmeal with cinnamon hint 
 

14) kolim / jawla 
 

15) chicken razala 
 



In [28]:
def return_by_rating(rating):
    data = ratings_with_food[ratings_with_food["rating"] == rating]
    result = data["name"].values.tolist()
    return {"By Rating":result}

In [29]:
ratings_with_food[ratings_with_food["rating"] == 1].head()

,user-id,food-id,rating,name,type,veg-non,describe
4,2.0,49.0,1.0,christmas tree pizza,Italian,veg,"pizza dough (2 boules), red pepper, red onion, basil pesto (purchased, homemade or vegan pesto), mozzarella cheese, kosher salt"
10,3.0,168.0,1.0,egg and garlic fried rice,Chinese,non-veg,"oil, garlic, spring onion , ginger, red chilli , egg, cooked rice, salt, black pepper powder, soya sauce"
16,3.0,299.0,1.0,kolim / jawla,Indian,veg,dried fish named kolim or jawla found in coastal maharashtra with onion and spices. usually eaten with bhakri or chapati
19,4.0,170.0,1.0,fried rice with soya chunks,Chinese,veg,"basmati rice, carrot, capsicum, beans, green peas, ginger, garlic, green chillies, bay leaf, cinnamon sticks, green cardamoms, clove, soya chunks, salt,"
22,4.0,21.0,1.0,apple rabdi,Dessert,veg,"apples, milk, sugar, green cardamoms, almonds (blanched), pistachios (blanched)"


In [30]:
return_by_rating(1)

{'By Rating': ['christmas tree pizza',
  'egg and garlic fried rice',
  'kolim / jawla',
  'fried rice with soya chunks',
  'apple rabdi',
  'almond and cranberry poha',
  'chocolate marquise',
  'prawn and litchi salad',
  'chicken razala',
  'chicken dragon',
  'lamb barley pot',
  'andhra crab meat masala',
  'sous-vide salmon tikka',
  'coconut mango oatmeal with cinnamon hint',
  'rice',
  'baba budan no',
  'carrot ginger soup',
  'holi special bhang pakode',
  'seafood rock filler',
  'almond and cranberry poha',
  'dates and nuts ladoo',
  'spaghetti with clams & crispy bread crumbs',
  'baked shankarpali ',
  'dahi lasooni chicken',
  'lamb barley pot',
  'spicy chicken curry',
  'spicy creamy kadai chicken',
  'brown rice',
  'watermelon and strawberry smoothie',
  'clotted cottage cheese croquettes',
  'californian breakfast benedict',
  'japanese fish stew',
  'garlic soya chicken',
  'rice in lamb stock',
  'japanese fish stew',
  'bhurji- egg',
  'chocolate fudge cookies'

In [31]:
data = ratings_with_food[ratings_with_food["rating"] == 10]
result = data["name"].values
result

array(['chicken and mushroom lasagna', 'al hachi chicken',
       'bread with tomatoes and olives', 'chicken quinoa biryani',
       'baked almond kofta', 'andhra pan fried pomfret', 'lamb korma',
       'chocolate kaju katli', 'cajun spiced turkey wrapped with bacon',
       'fruit cube salad', 'vegetable manchurian', 'filter coffee',
       'fish with jamun sauce', 'corn & jalapeno poppers',
       'french pork chop', 'christmas cake', 'instant rava dosa',
       'chicken sukka', 'cheese and ham roll', 'chilli fish',
       'vegetable bruschetta', 'chicken minced salad', 'christmas cake',
       'tandoori fish tikka', 'japanese curry arancini with barley salsa',
       'garlic and pinenut soup with burnt butter essence',
       'broccoli souffle', 'tandoori chicken', 'malabari fish curry',
       'christmas tree pizza', 'grilled lemon margarita',
       'surmai curry with lobster butter rice', 'apple and pear cake',
       'fish andlouse', 'andhra pan fried pomfret', 'bread chana bas